# Transformacion de los Datos para Modelado CLTV

Esta estapa es crucial para preparar el dataset para el modelado del CLTV con Redes Neuronales. Esta fase se realiza ya que no se pueden usar los datos tal cual como estan antes de esta etapa, se requiere transformar el registro de transacciones que esta en formato (filas por compras) a un formato de registros por cliente (una fila por cliente) estructurado en ventanas de tiempo.

In [26]:
import pandas as pd
from dateutil.relativedelta import relativedelta

df = pd.read_parquet('../data/processed/online_retail_II_cleaned.parquet')

fecha_inicio = df['invoicedate'].min().normalize()
fecha_fin = df['invoicedate'].max().normalize()

print(f"Rango de datos disponible: {fecha_inicio} a {fecha_fin}")

Rango de datos disponible: 2009-12-01 00:00:00 a 2010-11-30 00:00:00


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386732 entries, 0 to 386731
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   invoice      386732 non-null  string        
 1   stockcode    386732 non-null  string        
 2   description  386732 non-null  string        
 3   quantity     386732 non-null  int16         
 4   invoicedate  386732 non-null  datetime64[ns]
 5   price        386732 non-null  float32       
 6   customer_id  386732 non-null  Int32         
 7   country      386732 non-null  string        
dtypes: Int32(1), datetime64[ns](1), float32(1), int16(1), string(4)
memory usage: 18.8 MB


In [29]:
ventanas = []

# Definimos las fechas de inicio de cada ventana de observación
# Solo podemos empezar en meses donde tengamos 9 meses por delante (3 obs + 6 target)
# Fechas viables: 2009-12-01, 2010-01-01, 2010-02-01, 2010-03-01
start_dates = [pd.Timestamp('2009-12-01') + relativedelta(months=i) for i in range(4)]

for i, start_date in enumerate(start_dates):
    # Definir fechas clave para esta ventana
    split_date = start_date + relativedelta(months=3)  # Fin de X, Inicio de y
    end_window_date = split_date + relativedelta(months=6) # Fin de y

    # Validar que no nos pasemos del final de los datos reales
    if end_window_date > fecha_fin + pd.Timedelta(days=5): # Margen de error pequeño
        print(f"Ventana {i+1} descartada: termina en {end_window_date}, data termina en {fecha_fin}")
        break

    print(f"Procesando Ventana {i+1}: Obs[{start_date.date()} a {split_date.date()}) -> Target[{split_date.date()} a {end_window_date.date()}]")

    # 1. Filtrar Datos para X (Observación - 3 meses)
    df_X = df[(df['invoicedate'] >= start_date) & (df['invoicedate'] < split_date)]

    # 2. Filtrar Datos para y (Target - 6 meses)
    df_y = df[(df['invoicedate'] >= split_date) & (df['invoicedate'] <= end_window_date)]

    # 3. Construir Features (X)
    X_window = df_X.groupby('customer_id').agg({
        'invoicedate': lambda x: (split_date - x.max()).days, # Recency relativa al corte
        'Invoice': 'nunique',
        'TotalAmount': 'sum',
        'StockCode': 'nunique'
    }).reset_index()
    X_window.columns = ['customer_id', 'Rrecency', 'Frequency', 'Monetary', 'ProductVariety']

    # Feature Contextual: Añadimos el mes de inicio para que el modelo sepa la estacionalidad
    X_window['Month_Start'] = start_date.month

    # 4. Construir Target (y)
    y_window = df_y.groupby('customer_id')['TotalAmount'].sum().reset_index()
    y_window.columns = ['customer_id', 'Target_CLTV_6m']

    # 5. Unir y Etiquetar
    window_data = pd.merge(X_window, y_window, on='customer_id', how='left')
    window_data['Target_CLTV_6m'] = window_data['Target_CLTV_6m'].fillna(0)

    # Agregar a la lista maestra
    ventanas.append(window_data)

# Concatenar todas las ventanas en un solo gran dataset
df_model_rolling = pd.concat(ventanas, ignore_index=True)

print(f"\nDataset Final Generado: {df_model_rolling.shape[0]} registros totales.")
print(f"Ejemplos promedio por cliente: {df_model_rolling.shape[0] / df_model_rolling['customer_id'].nunique():.2f}")
df_model_rolling.head()

Procesando Ventana 1: Obs[2009-12-01 a 2010-03-01) -> Target[2010-03-01 a 2010-09-01]


KeyError: "Column(s) ['Invoice', 'StockCode', 'TotalAmount'] do not exist"